In [1]:
import json
import nltk
from nltk.corpus import stopwords
import re
from nltk import tokenize
from tqdm import tqdm
from nltk.corpus import words

In [2]:
st = stopwords.words('english')
common_words = words.words()

### Annotation dataset

Labels

1 - GENE

2 - DISEASE

In [3]:
with open("D:cours/textmining/genes.txt") as g:
    genes = set([ x.lower().strip() for x in g.readlines()])

In [4]:
st.append("ace")
st.append("large")
st.append("kit")
st.append("impact")
st.append("set")

In [9]:
genes_clean = []
for g in tqdm(genes):
    if g not in st and not g.isdigit() and g not in common_words:
        genes_clean.append(g)

100%|██████████| 16580/16580 [00:39<00:00, 419.05it/s]


In [10]:
len(genes_clean)

16278

In [6]:
len(genes_clean)

16415

In [11]:
with open("D:cours/textmining/Maladies.txt") as m:
    maladies = list(set([ x.lower().strip() for x in m.readlines()]))

Corpus

In [12]:
datas = open("D:cours/textmining/asthma.json").readlines()

### Create dataset by sentences

In [26]:
%%time
TRAIN_DATA_WORD=[]
train_i = 0
nb = 0
for line in tqdm(datas):
    text = json.loads(line)["ab"].lower()
    train_i += 1
    
    detected_disease = []
    for maladie in maladies:
        if maladie in text:
            detected_disease.append(maladie)
    
    for sentence in tokenize.sent_tokenize(text):
        if len(sentence) > 2:
            uhm = {"entities": []}
            
            for detect in detected_disease:
                if detect in sentence:
                    pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(detect), sentence)]
                    for qs in pmz:
                        uhm["entities"].append( (qs, qs+len(detect), "DISEASE")  )
            
            for n in tokenize.word_tokenize(sentence):
                if n in genes_clean:
                    pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n), sentence)]
                    for qs in pmz:
                        uhm["entities"].append( (qs, qs+len(n), "GENE")  )
            
            TRAIN_DATA_WORD.append( (sentence, uhm) )
    
    if train_i % 2000 == 0:
        with open("train/train_set_"+str(nb)+".json", "w") as t:
            t.write(json.dumps(TRAIN_DATA_WORD))
        nb += 1
        TRAIN_DATA_WORD=[]

100%|██████████| 44975/44975 [41:22<00:00, 18.12it/s]


Wall time: 41min 22s


In [24]:
alld = []
allg = []
for dg in TRAIN_DATA_WORD[0:1000]:
    if dg[1]["entities"] :
        for d  in dg[1]["entities"]:
            if d[2] == "DISEASE":
                allg.append(dg[0][d[0]:d[1]])
            if d[2] == "GENE":
#                 print()
#                 print(dg[0][d[0]:d[1]], d[2])
#                 if dg[0][d[0]:d[1]] == "set":
#                     print(dg)
                alld.append(dg[0][d[0]:d[1]])

In [25]:
set(alld)

{'app',
 'bcl2',
 'ccl21',
 'cd4',
 'chit1',
 'clps',
 'cs',
 'eed',
 'erbb4',
 'fev',
 'hr',
 'hta',
 'igf1r',
 'iqgap2',
 'myd88',
 'ndst1',
 'nhs',
 'nlrp3',
 'nms',
 'nps',
 'oxr1',
 'rhoa',
 'rnaset2',
 'sptlc2',
 'tgm3',
 'tlr2',
 'tlr7',
 'tlr8',
 'tnf',
 'traf6',
 'tslp'}

In [ ]:
# %%time
# TRAIN_DATA_WORD=[]
# train_i = 0
# nb = 0
# for line in tqdm(datas[0:2]):
#     text = json.loads(line)["ab"].lower()
#     train_i += 1
    
#     detected_disease = []
#     for maladie in maladies:
#         if maladie in text:
#             detected_disease.append(k)
    
#     for sentence in tokenize.sent_tokenize(text):
#         if len(sentence) > 2:
#             uhm = {"entities": []}
            
#             for detect in detected_disease:
#                 if detect in sentence:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "DISEASE")  )
            
            
#             for n in sentence.split(" "):
#                 n2 = n.replace(".", "").replace(",", "")
#                 if n2 in maladies:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "DISEASE")  )
#                 if n2 in genes_clean:
#                     pmz = [m.start() for m in re.finditer(r'\b%s\b' % re.escape(n2), sentence)]
#                     for qs in pmz:
#                         uhm["entities"].append( (qs, qs+len(n2), "GENE")  )
            
            
            
#             TRAIN_DATA_WORD.append( (sentence, uhm) )
    
# #     if train_i % 2000 == 0:
# #         with open("train/train_set_"+str(nb)+".json", "w") as t:
# #             t.write(json.dumps(TRAIN_DATA_WORD))
# #         nb += 1
# #         TRAIN_DATA_WORD=[]